# Vamos utilizar o mlflow para versionar a nossa model pipeline

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [3]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "Outcome"

## Utilizar o sqlite para fazer track das experiências

In [4]:
mlflow.set_tracking_uri("sqlite:///../../mlruns/mlflow.db")

## Fazer criar e fazer set da experiência "Diabetes Prediction Experiment"

Com este novo tracking_uri, se fizermos apenas `mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")` como no notebook anterior os artifacts vão ser guardados numa pasta de `mlruns` dentro da pasta `logistic_regression`.

Para termos os artifacts dentro da pasta `mlruns` que está na root da pasta `rumos` temos que especificar explicitamente o `artifact_location` e usar primeiro o `create_experiment` com este parâmetro, e só depois fazer o `set_experiment`. O `set_experiment` faz internamente um `create_experiment` caso a experiência com aquele nome não exista, mas como não recebe o parâmetro `artifact_location` neste caso temos que chamar explicitamente o `create_experiment` 

In [5]:
from pathlib import Path

artifact_location = Path("../../mlruns/db")

# criar a pasta ../../mlruns/db caso ela não exista
artifact_location.mkdir(parents=True, exist_ok=True)

try:
    mlflow.create_experiment(name="Diabetes Prediction Experiment", artifact_location=artifact_location.as_posix())
except mlflow.MlflowException:
    # experiência já foi criada, só precisamos de fazer set dela
    pass

mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")

<Experiment: artifact_location=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/random_forest/../../mlruns/db'), creation_time=1700517998255, experiment_id='1', last_update_time=1700517998255, lifecycle_stage='active', name='Diabetes Prediction Experiment', tags={}>

## Criar datasets

In [6]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

In [7]:
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name="Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name="Diabetes Test Dataset")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.ap

## Criar uma run

In [8]:

mlflow.start_run(run_name="Logistic Regression Pipeline Run")

<ActiveRun: >

## Guardar datasets, modelos, artefactos, métricas e parametros da run - e **registar a nossa model pipeline**

In [9]:
mlflow.log_param("seed", SEED)

42

In [10]:
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [11]:
X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

In [12]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3,173,78,39,185,33.8,0.970,31
1,6,134,80,37,370,46.2,0.238,46
2,5,104,74,0,0,28.8,0.153,48
3,1,139,46,19,83,28.7,0.654,22
4,5,137,108,0,0,48.8,0.227,37


In [15]:
# inicializar pipeline

lr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(random_state=SEED))
])


In [16]:
# fazer fit da pipeline

lr_pipeline.fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


Pipeline(steps=[('scaler', StandardScaler()),
                ('lr', LogisticRegression(random_state=42))])

In [17]:
# logar a pipeline

mlflow.sklearn.log_model(lr_pipeline, "lr_pipeline")

c:\Users\gilso\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
# fazer predict com a pipeline

y_preds = lr_pipeline.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [19]:
# medir accuracy do predict

accuracy_score(y_test, y_preds)

0.7105263157894737

In [22]:
# fazer grid search com a pipeline


In [21]:
# fazer grid search com a pipeline

lr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(random_state=SEED))
])

parameters = {"lr__C": [0.001, 0.01, 0.1, 1, 10, 100]}

# Assuming X_train and y_train are your training data
clf = GridSearchCV(lr_pipeline, parameters, cv=5)
clf.fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('lr',
                                        LogisticRegression(random_state=42))]),
             param_grid={'lr__C': [0.001, 0.01, 0.1, 1, 10, 100]})

In [23]:
# logar a grid search

mlflow.sklearn.log_model(clf, "grid_Search_CV")

In [24]:
# associar o melhor estimator encontrado pelo grid search a uma variavel

tunned_lr_pipeline = clf.best_estimator_

In [25]:
# registar o melhor estimator

mlflow.sklearn.log_model(tunned_lr_pipeline, "tunned_lr_pipeline", registered_model_name="logistic_regression")

Successfully registered model 'logistic_regression'.
Created version '1' of model 'logistic_regression'.


In [26]:
# ver os parametros do melhor estimator

tunned_lr_pipeline.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('lr', LogisticRegression(C=0.1, random_state=42))],
 'verbose': False,
 'scaler': StandardScaler(),
 'lr': LogisticRegression(C=0.1, random_state=42),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'lr__C': 0.1,
 'lr__class_weight': None,
 'lr__dual': False,
 'lr__fit_intercept': True,
 'lr__intercept_scaling': 1,
 'lr__l1_ratio': None,
 'lr__max_iter': 100,
 'lr__multi_class': 'auto',
 'lr__n_jobs': None,
 'lr__penalty': 'l2',
 'lr__random_state': 42,
 'lr__solver': 'lbfgs',
 'lr__tol': 0.0001,
 'lr__verbose': 0,
 'lr__warm_start': False}

In [27]:
# fazer predict com o melhor estimator

y_preds = tunned_lr_pipeline.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [28]:
# ter o accuracy score associado às predictions

acc = accuracy_score(y_test, y_preds)

In [29]:
# logar a accuracy

mlflow.log_metric("accuracy", acc)

## Terminar a run

In [30]:
mlflow.end_run()

## Opcional - Ver o modelo registado na UI do mlflow

A UI do mlflow permite ver de forma visual todos os modelos registados até ao momento, qual é a útlima versão do mesmo e a que runs estão associados.

Para correr a UI do mflow, tendo nós o modelo registado na db, temos que especificar esta db como a "backend store" em que temos os metados.

Para correr a UI do mflow é necessário executar, na Anaconda Prompt na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri sqlite:///./mlruns/mlflow.db`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`localhost:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Models` podem explorar os modelos que registaram.